# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Study data files
weather_metadata = "my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Gondanglegi,-7.75,112.01,76.91,82,91,3.38,ID,1604329669
1,1,Vao,-22.67,167.48,73.74,90,100,8.30,NC,1604329670
2,2,Rikitea,-23.12,-134.97,72.64,83,80,20.92,PF,1604329670
3,3,Bredasdorp,-34.53,20.04,73.40,64,0,18.34,ZA,1604329670
4,4,Bethel,41.37,-73.41,39.00,51,44,11.01,US,1604329670


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

# weatherdf = weatherdf.dropna()

hum_rate = weatherdf["Humidity"].astype(float)


In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weatherdf = weatherdf.loc[(weatherdf["Max Temp"] > 70) & (weatherdf["Max Temp"] < 80) & (weatherdf["Wind Speed"]<10) & (weatherdf["Cloudiness"] == 0), :]
ideal_weatherdf = ideal_weatherdf.dropna(how='any')

ideal_weatherdf.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,Hong Kong,22.29,114.16,73.0,63,0,5.99,HK,1604329417
37,37,Hoa Binh,20.81,105.34,71.6,64,0,5.82,VN,1604329678
39,39,Estrela,-28.06,-50.94,73.4,37,0,3.02,BR,1604329679
72,72,Azul,-36.78,-59.86,72.0,50,0,4.00,AR,1604329688
75,75,Hermanus,-34.42,19.23,77.0,62,0,8.01,ZA,1604329688


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame(ideal_weatherdf)
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,17,Hong Kong,22.29,114.16,73.0,63,0,5.99,HK,1604329417,
37,37,Hoa Binh,20.81,105.34,71.6,64,0,5.82,VN,1604329678,
39,39,Estrela,-28.06,-50.94,73.4,37,0,3.02,BR,1604329679,
72,72,Azul,-36.78,-59.86,72.0,50,0,4.00,AR,1604329688,
75,75,Hermanus,-34.42,19.23,77.0,62,0,8.01,ZA,1604329688,


In [8]:
# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Latitude']
    lng = row['Longitude']

    # update address key value
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

    # make request
    response = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    hotel_name = response.json()
#     pprint(hotel_name["results"][1]["name"])
    try:
        hotel_value = hotel_name['results'][1]['name']
        hotel_df.loc[index, "Hotel Name"] = hotel_value
        
    except:
        hotel_value=""
        
    print(hotel_value)
    print("------------")



Mini Hotel Central
------------
Hoa Dao Hotel
------------
Bar e Armazém Mazotti
------------
Cabañas Dulcinea
------------
Misty Waves Boutique Hotel
------------
Oyster Bay House Rental
------------
Orient Apartments
------------
Humacao Bed & Breakfast
------------
Mobilis
------------
dyd -mar Hotel
------------
Comisión Federal de Eléctricidad
------------
Sur Plaza Hotel
------------
ibis Styles Mt Isa Verona
------------
Credito Tres Arroyos
------------
NH La Perdiz
------------
Austral Hotel
------------
Palmar Hotel Casino
------------
Hotel Milenium
------------
Hotel Faro & Beach Club
------------
Le moulin vieux
------------


In [9]:
first_hotel_df = hotel_df.dropna(how='any')

In [10]:
first_hotel_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,17,Hong Kong,22.29,114.16,73.0,63,0,5.99,HK,1604329417,Mini Hotel Central
37,37,Hoa Binh,20.81,105.34,71.6,64,0,5.82,VN,1604329678,Hoa Dao Hotel
39,39,Estrela,-28.06,-50.94,73.4,37,0,3.02,BR,1604329679,Bar e Armazém Mazotti
72,72,Azul,-36.78,-59.86,72.0,50,0,4.00,AR,1604329688,Cabañas Dulcinea
75,75,Hermanus,-34.42,19.23,77.0,62,0,8.01,ZA,1604329688,Misty Waves Boutique Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in first_hotel_df.iterrows()]
locations2 = first_hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>